# Evaluation

- 앞에서 model.train() 모드로 변한 것처럼 평가 할 때는 model.eval()로 설정
    - Batch Normalization이나 Drop Out 같은 Layer들을 잠금
- autograd engine, 즉 backpropagatin이나 gradient 계산 등을 꺼서 memory usage를 줄이고 속도를 높임



In [1]:
import os
from glob import glob

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

import numpy as np

# Hypter Parameter

In [2]:
seed = 1

lr = 0.001
momentum = 0.5

batch_size = 64
test_batch_size = 64

epochs = 1
no_cuda = False
log_interval = 100

# Model

In [3]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1) # 입력(1,28,28) => conv1 => (20,24,24) => maxpool => (20,12,12)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)# 입력(20,12,12) => conv2 => (50,8,8) => maxpool => (50, 4, 4)
        self.fc1 = nn.Linear(4*4*50, 500)
        self.fc2 = nn.Linear(500, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4*4*50)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

In [4]:
save_dir = r'D:\DevRoot\DataSet\torch\dataset'

# Preprocess

In [5]:
torch.manual_seed(seed)

use_cuda = not no_cuda and torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}



train_loader = torch.utils.data.DataLoader(
    datasets.MNIST(save_dir, train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=batch_size, shuffle=True, **kwargs)




test_loader = torch.utils.data.DataLoader(
    datasets.MNIST(save_dir, train=False, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=test_batch_size, shuffle=True, **kwargs)

# Optimization

In [7]:
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)

In [8]:
params = list(model.parameters())

In [11]:
for param in params:
    print(param.size())

torch.Size([20, 1, 5, 5])
torch.Size([20])
torch.Size([50, 20, 5, 5])
torch.Size([50])
torch.Size([500, 800])
torch.Size([500])
torch.Size([10, 500])
torch.Size([10])


# Training

In [12]:
for epoch in range(1, epochs + 1):
    # Train Mode
    model.train()

    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()  # backpropagation 계산하기 전에 0으로 기울기 계산
        output = model(data)
        loss = F.nll_loss(output, target)  # https://pytorch.org/docs/stable/nn.html#nll-loss
        loss.backward()  # 계산한 기울기를 
        optimizer.step()  

        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.299826
Train Epoch: 1 [6400/60000 (11%)]	Loss: 2.216332
Train Epoch: 1 [12800/60000 (21%)]	Loss: 2.123600
Train Epoch: 1 [19200/60000 (32%)]	Loss: 1.936864
Train Epoch: 1 [25600/60000 (43%)]	Loss: 1.653543
Train Epoch: 1 [32000/60000 (53%)]	Loss: 1.178103
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.750639
Train Epoch: 1 [44800/60000 (75%)]	Loss: 0.734413
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.557860
Train Epoch: 1 [57600/60000 (96%)]	Loss: 0.445100


# Evaluation

- 앞에서 model.train() 모드로 변한 것처럼 평가 할 때는 model.eval()로 설정
    - Batch Normalization이나 Drop Out 같은 Layer들을 잠금

In [13]:
model.eval()  # 평가모드!

Net(
  (conv1): Conv2d(1, 20, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(20, 50, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=800, out_features=500, bias=True)
  (fc2): Linear(in_features=500, out_features=10, bias=True)
)

In [14]:
test_loss = 0
correct = 0

In [15]:
with torch.no_grad():   
     # evalucation 에선  batch normalization 이나 drop out 같은 것들을 잠금
    # no_grad() 는 back propagation 이나 gradient 등의 계산을 꺼서 memory usage 를 줄이고 속도 높임
    # 테스트 모드일때는 이를 꺼주는 것이 좋다!   
    
    data, target = next(iter(test_loader))
    data, target = data.to(device), target.to(device)
    output = model(data)
    
    # loss 계산.
    test_loss += F.nll_loss(output, target, reduction="sum").item()
            # reduction 을 안하면 batch size 별로, 데이터별로 따로 따로 계산하게 되는데
            # reduction=sum 을 해주면 하나의 스칼라 값으로 리턴해줌
            # 그것을 test_loss 에 누적 더해주는 것
    
    pred = output.argmax(dim=1, keepdim=True)  # 가장 강한 인덱스를 구함. 즉, 계산한 정답 예측값 꺼냄
    correct = pred.eq(target.view_as(pred)).sum().item()  # pred 과 target 이 같은가 판정
        # pred.eq(target.view_as(pred))  <-- 여기는 True / False 값 나옴
        #  .sum() 그것들을 다 더함.
        # .item() item() 에 담음       
    

In [16]:
test_loss

21.678577423095703

In [17]:
correct

60

In [19]:
output.shape

torch.Size([64, 10])

In [20]:
pred.shape    # keepdim=True 에 의해 output의 dimension 이 그대로 유지

torch.Size([64, 1])

In [21]:
target

tensor([9, 1, 9, 6, 3, 7, 9, 1, 0, 9, 1, 3, 6, 9, 2, 4, 7, 0, 9, 9, 7, 8, 4, 1,
        8, 5, 5, 0, 8, 9, 9, 3, 8, 6, 1, 7, 5, 3, 3, 8, 3, 5, 2, 4, 3, 9, 1, 7,
        8, 0, 3, 0, 9, 8, 1, 0, 0, 1, 6, 7, 5, 8, 8, 5])

In [22]:
target.shape

torch.Size([64])

In [24]:
target.view_as(pred)  # shape 를 맞추어 줌.
target.view_as(pred).shape

torch.Size([64, 1])

In [25]:
pred.eq(target.view_as(pred))

tensor([[ True],
        [ True],
        [False],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [False],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [False],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True

In [26]:
pred.eq(target.view_as(pred)).sum()

tensor(60)

In [27]:
pred.eq(target.view_as(pred)).sum().item()

60

In [28]:
pred.eq(target.view_as(pred)).sum().item() / 64  # 한 batch 에서 맞춘 확률

0.9375

In [29]:
test_loss  # loss 가 누적 되어 있다

21.678577423095703

In [31]:
# 그 평균값을 꺼내 보면
test_loss / len(test_loader.dataset)

0.00216785774230957

In [32]:
# 이상, 앞의 코드를 펼쳐 보았다

In [33]:
# 이제 loss 를 구했으니
# 하나로 묶어보자

In [34]:
model.eval()

test_loss = 0
correct = 0

with torch.no_grad():
    for data, target in test_loader:  # test 의 모든 batch 를 다 돌려 본다.
        data, target = data.to(device), target.to(device)
        output = model(data)
        
        test_loss += F.nll_loss(output, target, reduction='sum').item()  # 얼마나 틀렸는지 누적
        pred = output.argmax(dim=1, keepdim=True)  # 예측값
        correct += pred.eq(target.view_as(pred)).sum().item()  # 얼마나 맞혔는지도 누적
        
test_loss /= len(test_loader.dataset)  # loss 평균값

print('\nTest set : Average Loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100.0 * correct / len(test_loader.dataset)
))


Test set : Average Loss: 0.4530, Accuracy: 8846/10000 (88%)



---
# Train + Eval

In [35]:
# 한 epoch 를 train 하고 test 하는 과정들

In [36]:
epochs = 2

In [37]:
for epoch in range(1, epochs + 1):
    
    # Train Mode
    model.train()

    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()  # backpropagation 계산하기 전에 0으로 기울기 계산
        output = model(data)
        loss = F.nll_loss(output, target)  # https://pytorch.org/docs/stable/nn.html#nll-loss
        loss.backward()  # 계산한 기울기를 
        optimizer.step()  

        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
            
            
    # Evaluation Mode
    model.eval()

    test_loss = 0
    correct = 0

    with torch.no_grad():
        for data, target in test_loader:  # test 의 모든 batch 를 다 돌려 본다.
            data, target = data.to(device), target.to(device)
            output = model(data)

            test_loss += F.nll_loss(output, target, reduction='sum').item()  # 얼마나 틀렸는지 누적
            pred = output.argmax(dim=1, keepdim=True)  # 예측값
            correct += pred.eq(target.view_as(pred)).sum().item()  # 얼마나 맞혔는지도 누적

    test_loss /= len(test_loader.dataset)  # loss 평균값

    print('\nTest set : Average Loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
            test_loss, correct, len(test_loader.dataset),
            100.0 * correct / len(test_loader.dataset)
    ))

Train Epoch: 1 [0/60000 (0%)]	Loss: 0.295789
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.337058
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.312375
Train Epoch: 1 [19200/60000 (32%)]	Loss: 0.359015
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.295964
Train Epoch: 1 [32000/60000 (53%)]	Loss: 0.296687
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.244745
Train Epoch: 1 [44800/60000 (75%)]	Loss: 0.264813
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.212445
Train Epoch: 1 [57600/60000 (96%)]	Loss: 0.339521

Test set : Average Loss: 0.2901, Accuracy: 9167/10000 (92%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.387751
Train Epoch: 2 [6400/60000 (11%)]	Loss: 0.204017
Train Epoch: 2 [12800/60000 (21%)]	Loss: 0.281247
Train Epoch: 2 [19200/60000 (32%)]	Loss: 0.324143
Train Epoch: 2 [25600/60000 (43%)]	Loss: 0.143544
Train Epoch: 2 [32000/60000 (53%)]	Loss: 0.200259
Train Epoch: 2 [38400/60000 (64%)]	Loss: 0.301840
Train Epoch: 2 [44800/60000 (75%)]	Loss: 0.181387
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.390399
